<a href="https://colab.research.google.com/github/vamshap/PySpark-Challenges/blob/main/SwapSeats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import when, col, lag, lead
from pyspark.sql.window import Window

# write a code to swap seats consecutively if the total seats is odd then last seat would be the same seat
# Create Spark session
spark = SparkSession.builder.appName("SeatsTable").getOrCreate()

# Define the schema for the DataFrame
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("student", StringType(), True)
])

# Create the data
seats_data = [
    (1, 'Amit'),
    (2, 'Deepa'),
    (3, 'Rohit'),
    (4, 'Anjali'),
    (5, 'Neha'),
    (6, 'Sanjay'),
    (7, 'Priya')
]

# Create DataFrame
seats_df = spark.createDataFrame(data=seats_data, schema=schema)

# Add a column to identify odd and even IDs
seat_evenodd_df = seats_df.withColumn("oddEven", when(col("id") % 2 == 0, "even").otherwise("odd"))

# Define Window specifications
LagSpec = Window.orderBy("id")
LeadSpec = Window.orderBy("id")

# Add a column to swap seat IDs
sf = seat_evenodd_df.withColumn(
    "ExchangeSeat_ID",
    when(col("oddEven") == "even", lag("id").over(LagSpec))
    .when((col("oddEven") == "odd") & (lead("id").over(LeadSpec).isNull()), col("id"))
    .otherwise(lead("id").over(LeadSpec))
)

sf.show()

+---+-------+-------+---------------+
| id|student|oddEven|ExchangeSeat_ID|
+---+-------+-------+---------------+
|  1|   Amit|    odd|              2|
|  2|  Deepa|   even|              1|
|  3|  Rohit|    odd|              4|
|  4| Anjali|   even|              3|
|  5|   Neha|    odd|              6|
|  6| Sanjay|   even|              5|
|  7|  Priya|    odd|              7|
+---+-------+-------+---------------+

